<a href="https://colab.research.google.com/github/RAMGUNJAN/ERAV1_SESSION10/blob/main/ERAV1_Session10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms




In [2]:
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, transforms
import numpy as np
import random

# Load CIFAR-10 dataset with the required transform
transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Define CutOut transformation
class CutOut:
    def __init__(self, n_holes, length):
        self.n_holes = n_holes
        self.length = length

    def __call__(self, img):
        h, w = img.size(1), img.size(2)
        mask = np.ones((h, w), np.float32)

        for _ in range(self.n_holes):
            y = np.random.randint(h)
            x = np.random.randint(w)
            y1 = np.clip(y - self.length // 2, 0, h)
            y2 = np.clip(y + self.length // 2, 0, h)
            x1 = np.clip(x - self.length // 2, 0, w)
            x2 = np.clip(x + self.length // 2, 0, w)
            mask[y1: y2, x1: x2] = 0

        mask = torch.from_numpy(mask)
        img = img * mask.unsqueeze(0)

        return img

# Apply the CutOut transformation to the CIFAR-10 dataset
cutout_train_dataset = [(CutOut(n_holes=1, length=8)(image), label) for image, label in train_dataset]
cutout_test_dataset = [(CutOut(n_holes=1, length=8)(image), label) for image, label in test_dataset]

# Create DataLoader for training and testing
batch_size = 512
train_loader = torch.utils.data.DataLoader(cutout_train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(cutout_test_dataset, batch_size=batch_size, shuffle=False)


100%|██████████| 170498071/170498071 [00:02<00:00, 62828210.22it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [5]:
from google.colab import files
files.upload()

Saving model.py to model.py


{'model.py': b'# -*- coding: utf-8 -*-\n"""model.ipynb\n\nAutomatically generated by Colaboratory.\n\nOriginal file is located at\n    https://colab.research.google.com/drive/1JcL5Wgh2R5989I0cNfzdpKiJDtiBu_LP\n"""\n\nimport torch\nimport torch.nn as nn\nimport torch.nn.functional as F\n\nclass ResBlock(nn.Module):\n    def __init__(self, in_channels, out_channels):\n        super(ResBlock, self).__init__()\n        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1)\n        self.bn1 = nn.BatchNorm2d(out_channels)\n        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)\n        self.bn2 = nn.BatchNorm2d(out_channels)\n\n    def forward(self, x):\n        identity = x\n        out = F.relu(self.bn1(self.conv1(x)))\n        out = self.bn2(self.conv2(out))\n        out += identity\n        out = F.relu(out)\n        return out\n\nclass CustomResNet(nn.Module):\n    def __init__(self, num_classes=10):\n        super(

In [11]:
import model
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = model.CustomResNet().to(device)
summary(model, input_size=(3, 32, 32))

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,792
       BatchNorm2d-2           [-1, 64, 32, 32]             128
              ReLU-3           [-1, 64, 32, 32]               0
            Conv2d-4          [-1, 128, 32, 32]          73,856
         MaxPool2d-5          [-1, 128, 16, 16]               0
       BatchNorm2d-6          [-1, 128, 16, 16]             256
              ReLU-7          [-1, 128, 16, 16]               0
            Conv2d-8          [-1, 128, 16, 16]         147,584
       BatchNorm2d-9          [-1, 128, 16, 16]             256
           Conv2d-10          [-1, 128, 16, 16]         147,584
      BatchNorm2d-11          [-1, 128, 16, 16]             256
         ResBlock-12          [-1, 128, 16, 16]               0
           Conv2d-13          [-1, 256, 16, 16]         295,168
        MaxPool2d-14            [-

In [12]:
from tqdm import tqdm
import torch.nn.functional as F

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct = 0
    processed = 0
    for batch_idx, (data, target) in enumerate(pbar):
        # get samples
        data, target = data.to(device), target.to(device)

        # Init
        optimizer.zero_grad()

        # Predict
        y_pred = model(data)

        # Calculate loss
        loss = F.cross_entropy(y_pred, target)
        train_losses.append(loss.item())

        # Backpropagation
        loss.backward()
        optimizer.step()

        # Update pbar-tqdm
        pred = y_pred.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)

        pbar.set_description(desc=f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
        train_acc.append(100*correct/processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    test_acc.append(100. * correct / len(test_loader.dataset))

In [14]:
import model
import torch.optim as optim
import torch.nn as nn
from torch.optim.lr_scheduler import LRScheduler

# Define a simple learning rate range test scheduler
class LrRangeFinder(LRScheduler):
    def __init__(self, optimizer, lr_min, lr_max, total_epochs, steps_per_epoch):
        self.lr_min = lr_min
        self.lr_max = lr_max
        self.total_epochs = total_epochs
        self.steps_per_epoch = steps_per_epoch
        super(LrRangeFinder, self).__init__(optimizer)

    def get_lr(self):
        epoch = self.last_epoch + 1
        lr = self.lr_min + (epoch / self.total_epochs) * (self.lr_max - self.lr_min)
        return [lr] * len(self.base_lrs)

# Initialize the model and device as before
model = model.CustomResNet().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Set LRMIN and LRMAX values
LRMIN = 1e-6
LRMAX = 0.1

# Define the total number of epochs
EPOCHS = 15

# Define the number of steps per epoch
steps_per_epoch = len(train_loader)

# Training loop with learning rate range test
lr_scheduler = LrRangeFinder(optimizer, LRMIN, LRMAX, EPOCHS, steps_per_epoch)

for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    lr_scheduler.step()  # Update the learning rate for each epoch

    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


EPOCH: 0


Loss=1.7215200662612915 Batch_id=97 Accuracy=22.06: 100%|██████████| 98/98 [00:20<00:00,  4.83it/s]



Test set: Average loss: 1.9046, Accuracy: 3199/10000 (31.99%)

EPOCH: 1


Loss=1.2857106924057007 Batch_id=97 Accuracy=45.80: 100%|██████████| 98/98 [00:20<00:00,  4.86it/s]


...


Test set: Average loss: 1.1051, Accuracy: 7332/10000 (73.32%)

EPOCH: 13


Loss=0.25916415452957153 Batch_id=97 Accuracy=91.42: 100%|██████████| 98/98 [00:20<00:00,  4.77it/s]



Test set: Average loss: 1.5141, Accuracy: 6714/10000 (67.14%)

EPOCH: 14


Loss=0.31313252449035645 Batch_id=97 Accuracy=91.26: 100%|██████████| 98/98 [00:20<00:00,  4.77it/s]



Test set: Average loss: 1.0339, Accuracy: 7445/10000 (74.45%)

